# Safe to eat or deadly poisonous?
### An analysis on mushroom classification by Lorenzo Santolini

#### Code snippet for google colab
This is a little code to import automatically the dataset into google colab. Provide your kaggle's API key (profile section) when file requested

In [1]:

# Little code snippet to import on Google Colab the dataset
'''
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

# Insert here your kaggle API key
from google.colab import files
files.upload()

!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d uciml/mushroom-classification
!unzip mushroom-classification.zip
!ls
'''


'\n!pip install -U -q kaggle\n!mkdir -p ~/.kaggle\n\n# Insert here your kaggle API key\nfrom google.colab import files\nfiles.upload()\n\n!cp kaggle.json ~/.kaggle/\n!kaggle datasets download -d uciml/mushroom-classification\n!unzip mushroom-classification.zip\n!ls\n'

In [2]:
# Constants
PLOTLY_COLORS = ['#140DFF', '#FF0DE2']
COLOR_PALETTE = ['#FF0DE2', '#BD0CE8', '#8B00FF', '#4D0FE8', '#140DFF']
COLORSCALE_HEATMAP = [         [0.0, 'rgb(70,0,252)'], 
                [0.1111111111111111, 'rgb(78,0,252)'], 
                [0.2222222222222222, 'rgb(90,0,252)'], 
                [0.3333333333333333, 'rgb(110,0,248)'], 
                [0.4444444444444444, 'rgb(130,0,238)'], 
                [0.5555555555555556, 'rgb(145,0,228)'], 
                [0.6666666666666666, 'rgb(166,0,218)'], 
                [0.7777777777777778, 'rgb(187,0,213)'], 
                [0.8888888888888888, 'rgb(200,0,202)'], 
                               [1.0, 'rgb(210,0,191)']]


## Introduction
This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family Mushroom. Each species is identified as edible or poisonous. Rows are composed by 23 different fields, each one of them identifying a specific charateristic:

- Class: poisonous=p, edible=e
- Cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
- Cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s
- Cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
- Bruises: bruises=t, no=f
- Odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
- Gill-attachment: attached=a, descending=d, free=f, notched=n
- Gill-spacing: close=c, crowded=w, distant=d
- Gill-size: broad=b, narrow=n
- Gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u,red=e, white=w, yellow=y
- Stalk-shape: enlarging=e, tapering=t
- Stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
- Stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s
- Stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s
- Stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
- Stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
- Veil-type: partial=p, universal=u
- Veil-color: brown=n, orange=o, white=w, yellow=y
- Ring-number: none=n, one=o, two=t
- Ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
- Spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
- Population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
- Habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

This analysis was conducted in Python 3.7.1 using Jupyter Notebook allows you to combine code, comments, multimedia, and visualizations in an interactive document — called a notebook, naturally — that can be shared, re-used, and re-worked. In addition, the following packages were used:

- sklearn
- pandas
- numpy
- plotly


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

import plotly
import plotly.plotly as py
from plotly.plotly import plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

plotly.tools.set_credentials_file(username='modusV', api_key='OBKKnTR2vYTeKIOKtRU6')

## Dataset analysis and preprocessing
Let's start importing the data:

In [4]:
# Load the dataset
dataset = pd.read_csv("./Input/mushrooms.csv")
# dataset = pd.read_csv("./mushrooms.csv")
print("The dataset has %d rows and %d columns." % dataset.shape)

The dataset has 8124 rows and 23 columns.


Now we will look at the dataset to understand what are the different fields and their types:

In [5]:
# Count number of classes for classification
print(f"There are {dataset['class'].unique().size} different classes:"
      f"\n {dataset['class'].unique().tolist()}")

# Count number of unique data for every column
print(f"Unique values for every field: \n{dataset.nunique()}")


There are 2 different classes:
 ['p', 'e']
Unique values for every field: 
class                        2
cap-shape                    6
cap-surface                  4
cap-color                   10
bruises                      2
odor                         9
gill-attachment              2
gill-spacing                 2
gill-size                    2
gill-color                  12
stalk-shape                  2
stalk-root                   5
stalk-surface-above-ring     4
stalk-surface-below-ring     4
stalk-color-above-ring       9
stalk-color-below-ring       9
veil-type                    1
veil-color                   4
ring-number                  3
ring-type                    5
spore-print-color            9
population                   6
habitat                      7
dtype: int64


In [6]:
# See data types 
print(f"Data types: \n{dataset.head(5)}")

# All columns 
print(", ".join(str(a) for a in dataset.columns))

Data types: 
  class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color   ...   stalk-surface-below-ring  \
0            c         n          k   ...                          s   
1            c         b          k   ...                          s   
2            c         b          n   ...                          s   
3            c         n          n   ...                          s   
4            w         b          k   ...                          s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w  

From the above snippet we can notice that the fields are all string values; converting them to numeric values can make our analysis much easier. We will take care of this in the next phase.

## Preprocessing
We need to pre-process our data to 


In [7]:
n_columns_original = len(dataset.columns)
to_drop = [col for col in dataset.columns if dataset[col].nunique() == 1]
dataset.drop(to_drop, axis=1, inplace=True)

print(f"{n_columns_original - len(dataset.columns)} not significant columns have been removed")


#### Handling missing values

1 not significant columns have been removed


In [8]:
# Check if any field is null
if dataset.isnull().any().any():
    print("There are some null values")
else:
    print("There are no null values")

There are no null values


It may seem that we have no missing value from the previous analysis, but if we look better,from the data description we can notice that in the field stalk-root there are some missing values, marked with the question mark; let's count how many of them there are:


In [9]:
print("There are " + str((dataset['stalk-root'] == "?").sum()) + " missing values in stalk-root column")

There are 2480 missing values in stalk-root column


When we find missing values in a dataset, there are some of the approaches that can be considered:

1. Delete all rows containing a missing value
2. Substitute with a constant value that has meaning within the domain, such as 0, distinct from all other values.
3. Substitute with a value from another randomly selected record.
4. Substitute with mean, median or mode value for the column.
5. Substitute with a value estimated by another predictive model.

It is evident from the `dataset.head()` function that our fileds are composed by all string values. Given the fact that we would need to translate in any case every field to a numeric one, to better display them in graphs, a simple approach is to keep the missing data as a peculiar number different from the others, and simply apply the transformation as they were present.

We will use the LabelEncoder from the sklearn library, which allows us to perform this mapping:



In [14]:
def preprocess(dataset):
  pre_data = dataset.copy()
  labelEncoder = LabelEncoder()
  for column in dataset.columns:
    dataset[column] = labelEncoder.fit_transform(dataset[column])
  return pre_data, le

le = 0
pre_data, le = preprocess(dataset)
# Check new data
pre_data.head(5)


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,2,7,7,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,2,7,7,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,2,7,7,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,2,7,7,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,2,7,7,2,1,0,3,0,1


Now let's check if there is any column not useful for classification, 
where the values are all the same (zero variance)



In [15]:
# Check new labels
print(pre_data.groupby('class').size())

class
0    4208
1    3916
dtype: int64


We can notice that data have been transformed, and now the labels are represented with a 0/1 integer value. 
Now we can look deeper into some statistical details about the dataset, using the `dataset.describe` command on our pandas DataFrame dataset. The output describes:

- count: number of samples (rows)
- mean: the mean of the attribute among all samples
- std: the standard deviation of the attribute
- min: the minimal value of the attribute
- 25%: the lower percentile
- 50%: the median
- 75%: the upper percentile
- max: the maximal value of the attribute

In [16]:
# Get insights on the dataset
dataset.describe()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,...,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,0.482029,3.348104,1.827671,4.504677,0.415559,4.144756,0.974151,0.161497,0.309207,4.810684,...,1.575086,1.603644,5.816347,5.794682,1.965534,1.069424,2.291974,3.596750,3.644018,1.508616
std,0.499708,1.604329,1.229873,2.545821,0.492848,2.103729,0.158695,0.368011,0.462195,3.540359,...,0.621459,0.675974,1.901747,1.907291,0.242669,0.271064,1.801672,2.382663,1.252082,1.719975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,...,1.000000,1.000000,6.000000,6.000000,2.000000,1.000000,0.000000,2.000000,3.000000,0.000000
50%,0.000000,3.000000,2.000000,4.000000,0.000000,5.000000,1.000000,0.000000,0.000000,5.000000,...,2.000000,2.000000,7.000000,7.000000,2.000000,1.000000,2.000000,3.000000,4.000000,1.000000
75%,1.000000,5.000000,3.000000,8.000000,1.000000,5.000000,1.000000,0.000000,1.000000,7.000000,...,2.000000,2.000000,7.000000,7.000000,2.000000,1.000000,4.000000,7.000000,4.000000,2.000000
max,1.000000,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,...,3.000000,3.000000,8.000000,8.000000,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


In [17]:
y = dataset["class"].value_counts()
print(y)
class_dict = ["edible", "poisonous"]


0    4208
1    3916
Name: class, dtype: int64


This is the class distribution:

In [18]:
data = [go.Bar(
            x=class_dict,
            y=y,
            marker=dict(
            color=PLOTLY_COLORS),
    )]

layout = go.Layout(title="Class distribution",
                   autosize=False,
                   width=400,
                   height=400,
                   yaxis=dict(
                        title='N. samples',
                    ),
                   )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='color-bar')


/Users/lorenzosantolini/miniconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead




At this point we can analyze the distribution of our data using a boxplot. 
A boxplot is a standardized way of displaying the distribution of data based on a 
five number summary (“minimum”, first quartile (Q1), median, third quartile (Q3), and “maximum”). 
It can tell you about your outliers and what their values are. 
It can also tell you if your data is symmetrical, how tightly your data is grouped, 
and if and how your data is skewed.
The information that we can find in a box plot are:

- **median** (Q2/50th Percentile): the middle value of the dataset.
- **first quartile** (Q1/25th Percentile): the middle number between the smallest number (not the “minimum”) and the median of the dataset.
- **third quartile** (Q3/75th Percentile): the middle value between the median and the highest value (not the “maximum”) of the dataset.
- **interquartile range** (IQR): 25th to the 75th percentile.
- **outliers** (shown as green circles)
- **maximum**: Q3 + 1.5*IQR
- **minimum**: Q1 -1.5*IQR

It makes no sense showing binary or with few different values fields, so we are going to filter them before plotting.


In [19]:

def create_box(type, data, col, visible=False):
    if type == "edible":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    return go.Box(
        y = data[col],
        name = type,
        marker=dict(color = c),
        visible=visible,
    )

edible = pre_data[pre_data["class"] == 0]
poisonous = pre_data[pre_data["class"] == 1]
box_features = [col for col in pre_data.columns if ((col != 'class') and (dataset[col].nunique() > 5))]

active_index = 0
box_edible = [(create_box("edible", edible, col, False) if i != active_index 
               else create_box("edible", edible, col, True)) 
              for i, col in enumerate(box_features)]

box_poisonous = [(create_box("poisonous", poisonous, col, False) if i != active_index 
               else create_box("poisonous", poisonous, col, True)) 
              for i, col in enumerate(box_features)]

data = box_edible + box_poisonous
n_features = len(box_features)
steps = []

for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data)],
        label = box_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='box_slider')



From the boxplot above, we can see that the colour and the shape of the 
cap are not an effective parameter to decide whether a mushroom is poisonous or edible, 
because their plots are very similar (same median and very close distribution). 
The gill color instead, is more significant; 



Let's investigate the correlation between variables:


In [20]:
correlation_matrix = pre_data.corr(method='pearson')

trace = go.Heatmap(
    z=correlation_matrix.values.tolist(), 
    x=correlation_matrix.columns, 
    y=correlation_matrix.columns, 
    colorscale=COLORSCALE_HEATMAP,
    zmin=-1,
    zmax=1)
    

data=[trace]

layout = go.Layout(
    title='Heatmap of columns correlation',
    autosize=False,
    width=850,
    height=700,
    yaxis=go.layout.YAxis(automargin=True),
    xaxis=dict(tickangle=40),
    margin=go.layout.Margin(l=0, r=200, b=200, t=80)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap4')



TODO: Give some impressions on the heatmap


In [21]:

def create_hist(type, data, col, visible=False):
    if type == "edible":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    return go.Histogram(
        y = data[col],
        name = type,
        marker=dict(color = c),
        visible=visible,
    )

hist_features = [col for col in pre_data.columns if (col != 'class')]

active_index = 0
hist_edible = [(create_hist("edible", edible, col, False) if i != active_index 
               else create_hist("edible", edible, col, True)) 
              for i, col in enumerate(hist_features)]

hist_poisonous = [(create_hist("poisonous", poisonous, col, False) if i != active_index 
               else create_hist("poisonous", poisonous, col, True)) 
              for i, col in enumerate(hist_features)]

data = hist_edible + hist_poisonous
n_features = len(hist_features)
steps = []

for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data)],
        label = hist_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='hist_slider')
